<a href="https://colab.research.google.com/github/Ashish265/AI-ML-DL/blob/master/Designing_a_convolutional_network_from_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Importing the required libraries

import numpy as np
import pandas as pd
from tqdm import tqdm
from keras.datasets import mnist

Using TensorFlow backend.


In [2]:
# Loading dataset
(X_train,y_train),(X_test,y_test) = mnist.load_data()

11493376/11490434 [==============================] - 2s 0us/step


In [3]:
# Selecting a subset of data
X_train = X_train[:200]
y = y_train[:200]

In [4]:
X = X_train.T
X = X/255

In [5]:
y.resize((200,1))
y =y.T

In [6]:
pd.Series(y[0]).value_counts()

1    26
9    23
7    21
4    21
3    21
0    21
2    20
6    19
8    15
5    13
dtype: int64

In [7]:
#Converting into a binary class

for i in range(y.shape[1]):
  if y[0][i] > 4 :
    y[0][i] = 1
  else:
    y[0][i] = 0

pd.Series(y[0]).value_counts()


0    109
1     91
dtype: int64

In [8]:
# initializing filter
f=np.random.uniform(size=(3,5,5))
f = f.T

print('Filter 1', '\n', f[:,:,0], '\n')
print('Filter 2', '\n', f[:,:,1], '\n')
print('Filter 3', '\n', f[:,:,2], '\n')

Filter 1 
 [[0.19227044 0.34795763 0.81966541 0.02567578 0.55373436]
 [0.31031165 0.3031465  0.91685216 0.32710426 0.95352411]
 [0.06926033 0.40013485 0.57930773 0.30921582 0.65368938]
 [0.54716064 0.83455147 0.47304455 0.39246499 0.92236486]
 [0.21964524 0.62945335 0.75725833 0.75665004 0.40162004]] 

Filter 2 
 [[0.52816774 0.62524068 0.65157892 0.79842308 0.45031961]
 [0.39796777 0.58845202 0.5251121  0.05325141 0.42239549]
 [0.26465792 0.60835072 0.53512776 0.00951851 0.19101659]
 [0.53010825 0.22235439 0.40194856 0.35093311 0.92465465]
 [0.33855242 0.98600211 0.59322125 0.47684936 0.59231381]] 

Filter 3 
 [[0.91489382 0.92261959 0.26591905 0.32372181 0.02111395]
 [0.84074011 0.69852432 0.43377663 0.71883807 0.25282585]
 [0.42344339 0.82257488 0.28186003 0.77017614 0.65777085]
 [0.14458096 0.2521596  0.74810005 0.85061612 0.12033501]
 [0.29838352 0.61537599 0.66544196 0.6588022  0.34447551]] 



In [9]:
X.shape, y.shape, f.shape

((28, 28, 200), (1, 200), (5, 5, 3))

In [10]:

# Generating patches from images
new_image = []

# for number of images
for k in range(X.shape[2]):
    # sliding in horizontal direction
    for i in range(X.shape[0]-f.shape[0]+1):
        # sliding in vertical direction
        for j in range(X.shape[1]-f.shape[1]+1):
            new_image.append(X[:,:,k][i:i+f.shape[0],j:j+f.shape[1]])
            
# resizing the generated patches as per number of images
new_image = np.array(new_image)
new_image.resize((X.shape[2],int(new_image.shape[0]/X.shape[2]),new_image.shape[1],new_image.shape[2]))
new_image.shape

(200, 576, 5, 5)

In [11]:
# number of features in data set
s_row = X.shape[0] - f.shape[0] + 1
s_col = X.shape[1] - f.shape[1] + 1
num_filter = f.shape[2]

inputlayer_neurons = (s_row)*(s_col)*(num_filter)
output_neurons = 1 

# initializing weight
wo=np.random.uniform(size=(inputlayer_neurons,output_neurons))

In [12]:
# defining the Sigmoid Function
def sigmoid (x):
    return 1/(1 + np.exp(-x))

# derivative of Sigmoid Function
def derivatives_sigmoid(x):
    return x * (1 - x)

In [13]:
# generating output of convolution layer
filter_output = []
# for each image
for i in range(len(new_image)):
    # apply each filter
    for k in range(f.shape[2]):
        # do element wise multiplication
        for j in range(new_image.shape[1]):
            filter_output.append((new_image[i][j]*f[:,:,k]).sum()) 

filter_output = np.resize(np.array(filter_output), (len(new_image),f.shape[2],new_image.shape[1]))

# applying activation over convolution output
filter_output_sigmoid = sigmoid(filter_output)

filter_output.shape, filter_output_sigmoid.shape

((200, 3, 576), (200, 3, 576))

In [14]:
# generating input for fully connected layer
filter_output_sigmoid = filter_output_sigmoid.reshape((
filter_output_sigmoid.shape[0],
filter_output_sigmoid.shape[1]*filter_output_sigmoid.shape[2]
))

filter_output_sigmoid = filter_output_sigmoid.T

# Linear trasnformation for fully Connected Layer
output_layer_input=np.dot(wo.T,filter_output_sigmoid)
output_layer_input=(output_layer_input - np.average(output_layer_input))/np.std(output_layer_input)

# activation function
output = sigmoid(output_layer_input)

In [16]:
#Error
error = np.square(y-output)/2

#Error w.r.t Output (Gradient)
error_wrt_output = -(y-output)

#Error w.r.t sigmoid transformation (output_layer_input)
output_wrt_output_layer_input=output*(1-output)

#Error w.r.t weight
output_wrt_w=filter_output_sigmoid

In [18]:
#Error w.r.t sigmoid output
output_layer_input_wrt_filter_output_sigmoid = wo.T

#Error w.r.t sigmoid transformation
filter_output_sigmoid_wrt_filter_output = filter_output_sigmoid * (1-filter_output_sigmoid)

# calculating derivatives for backprop convolution
error_wrt_filter_output = np.dot(output_layer_input_wrt_filter_output_sigmoid.T,
error_wrt_output * output_wrt_output_layer_input) * filter_output_sigmoid_wrt_filter_output

error_wrt_filter_output = np.average(error_wrt_filter_output, axis=1)

error_wrt_filter_output = np.resize(error_wrt_filter_output,
(X.shape[0]-f.shape[0]+1,X.shape[1]-f.shape[1]+1, f.shape[2]))

In [19]:

filter_update = []
for i in range(f.shape[2]):
    for j in range(f.shape[0]):
        for k in range(f.shape[1]):            
            temp = 0
            spos_row = j
            spos_col = k
            epos_row = spos_row + s_row
            epos_col = spos_col + s_col
            for l in range(X.shape[2]):
                temp = temp + (X[spos_row:epos_row,spos_col:epos_col,l]*error_wrt_filter_output[:,:,i]).sum()
            filter_update.append(temp/X.shape[2])  

filter_update_array = np.array(filter_update)
filter_update_array = np.resize(filter_update_array,(f.shape[2],f.shape[0],f.shape[1]))

In [21]:
lr=0.1

In [22]:
for i in range(f.shape[2]):
    f[:,:,i] = f[:,:,i] - lr*filter_update_array[i]